In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
                                        ## (64,64,3) -- one kind of Ip format for color imgs
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten())
model.add(Dense(units = 128, activation = 'relu')) ## last hidden layer before the Op layer
model.add(Dense(units = 1, activation = 'sigmoid'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Pre-processing bfr giving it to the model
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)
val_datagen = ImageDataGenerator(rescale = 1./255)

## Training
training_set = train_datagen.flow_from_directory('FaceMask Dataset/Train',target_size = (64, 64),batch_size = 8,class_mode = 'binary')
val_set = val_datagen.flow_from_directory('FaceMask Dataset/Val',target_size = (64, 64),batch_size = 8,class_mode = 'binary')

model.fit_generator(training_set,steps_per_epoch = 10,epochs = 50,validation_data = val_set,validation_steps = 2)

model_json = model.to_json()
with open("Maskmodel.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("Maskmodel.h5")
print("Saved model to disk")


Found 85 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


C:\Users\hp\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
10/10 [==============================] - 4s 376ms/step - loss: 3.1273 - accuracy: 0.5195 - val_loss: 0.7268 - val_accuracy: 0.4375
Epoch 2/50
10/10 [==============================] - 3s 346ms/step - loss: 0.7127 - accuracy: 0.4935 - val_loss: 0.6940 - val_accuracy: 0.5000
Epoch 3/50
10/10 [==============================] - 3s 358ms/step - loss: 0.7116 - accuracy: 0.5065 - val_loss: 0.6910 - val_accuracy: 0.5000
Epoch 4/50
10/10 [==============================] - 4s 353ms/step - loss: 0.6874 - accuracy: 0.5325 - val_loss: 0.6851 - val_accuracy: 0.6250
Epoch 5/50
10/10 [==============================] - 3s 362ms/step - loss: 0.7083 - accuracy: 0.5455 - val_loss: 0.6797 - val_accuracy: 0.6250
Epoch 6/50
10/10 [==============================] - 3s 366ms/step - loss: 0.6498 - accuracy: 0.7532 - val_loss: 0.6790 - val_accuracy: 0.5000
Epoch 7/50
10/10 [==============================] - 3s 291ms/step - loss: 0.6167 - accuracy: 0.7662 - val_loss: 0.7025 - val_accuracy: 0.5000
Epoch 

In [1]:
from tensorflow.keras.models import model_from_json
import numpy as np
from tensorflow.keras.preprocessing import image

json_file = open('Maskmodel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("Maskmodel.h5")
print("Loaded model from disk")


import cv2

## importing the algo and loading the algo into CV using OpenCv 
haar_cascade = cv2.CascadeClassifier(r'C:\Users\hp\anaconda3\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml') 
cam = cv2.VideoCapture(0)
labels_dict = {0:'MASKED',1:'NOT MASKED'}
color_dict = {0:(0,255,0),1:(0,0,255)}

while True:
    _,img = cam.read()
    greyImg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face = haar_cascade.detectMultiScale(greyImg,1.3,4)  ## passing haar cascade frontal face algo with the greyscaled img
                                                         ## face will contain values of x,y,w,h --> coordinates of the face
    for x,y,w,h in face:
        face_img = greyImg[y:y+w,x:x+w]
        resized = cv2.resize(face_img,(64,64))
        normalized = resized/255.0
        reshaped = np.reshape(normalized,(64,64,3))
        result = model.predict(reshaped)
        
        label = np.argmax(result,axis=1)[0]
                
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)    #2
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1) 
        cv2.putText(img,labels_dict[label],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (225,255,255), 2)

        
    cv2.imshow("CameraFeed",img)
    key = cv2.waitKey(10)   
    if key == 27:    ## pressing 'esc' key
        break
    
cam.release()
cv2.destroyAllWindows()

Loaded model from disk


ValueError: cannot reshape array of size 4096 into shape (64,64,3)